In [2]:
import tensorflow as tf
import keras
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
!wget https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt

In [4]:
with open("input.txt", 'r', encoding='utf-8') as f:
  text = f.read()

In [7]:
text = text.lower()

In [ ]:
# Remove unnecessary chars

In [15]:
def remove_special_chars(text):
  """Removes all characters except alphanumeric characters (a-z), space, ?, and \n from a text string.

  Args:
      text: The input text string.

  Returns:
      The text string with all special characters removed.
  """

  allowed_chars = set("abcdefghijklmnopqrstuvwxyz \n:,.?")
  filtered_text = ''.join([char for char in text if char in allowed_chars])
  return filtered_text


In [16]:
text = remove_special_chars(text)


In [17]:
# create vocab from text and token mapping
vocab_text = sorted(list(set(text)))

ctoi = {k:i for i,k in enumerate(vocab_text)}
ctoi
itoc = {i:k for i,k in enumerate(vocab_text)}
vocab_size = len(vocab_text)


In [18]:
vocab_size

32

In [21]:
# tokenize routines
encode = lambda x: [ctoi[c] for c in x ]
decode = lambda x: ''.join([itoc[c] for c in x])

In [23]:
# tokenize the text
etext = encode(text)

In [24]:
# vectorize text
vtext = tf.constant(etext)

In [48]:
# we already have vocab_size. Lets get some more hyperparams
block_size = 8
batch_size = 32
embed_size = 32
num_heads = 4


In [26]:
# create Train and Val data
n = int(0.9 * len(vtext))
train_data = vtext[:n]
val_data = vtext[n:]

In [27]:
tf.random.set_seed(1234)
def get_batch(split):
  data = train_data if split == 'train' else val_data
  random_int = tf.random.uniform(shape=(batch_size, ), maxval=len(data) - block_size, minval=0, dtype=tf.int32)
  x = tf.stack([data[i:i+block_size] for i in random_int])
  y = tf.stack([data[i+1:i+block_size+1] for i in random_int])
  return x, y

In [28]:
# Single Head implementing Attention.
class Head(tf.keras.layers.Layer):
  def __init__(self, head_size):
    super(Head, self).__init__()
    self.key = tf.keras.layers.Dense(head_size) #n_embed
    self.query = tf.keras.layers.Dense(head_size)
    self.value = tf.keras.layers.Dense(head_size)
    self.tril = tf.ones((block_size, block_size))
    self.tril = tf.linalg.band_part(self.tril, -1, 0)
    self.dropout_layer = tf.keras.layers.Dropout(0.2)



  def call(self, x):
    # Perform custom operations on the input
    B,T,C = x.shape
    k=self.key(x)
    q=self.query(x)
    k_t = tf.transpose(k, perm=(0,2,1))

    wei = q @ k_t * C**-0.5

    mask = tf.equal(self.tril, 0)
    replace_value = -1e1000
    wei = tf.where(mask, replace_value, wei)
    wei = tf.nn.softmax(wei, axis=-1)
    wei = self.dropout_layer(wei, training=True)

    v = self.value(x)
    out = wei @ v

    return out

In [33]:

class MultiHead(tf.keras.layers.Layer):
  def __init__(self, num_heads, head_size):
    super(MultiHead, self).__init__()
    self.num_heads = num_heads
    self.head_size = head_size
    #self.heads = tf.keras.layers.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.heads = []  # Use a list instead of ModuleList
    for _ in range(num_heads):
      self.heads.append(Head(head_size))

    self.proj = tf.keras.layers.Dense(head_size*num_heads)


  def call(self, x):
    head_outputs = []
    for head in self.heads:
      out = head(x)  # Apply each head to the input
      head_outputs.append(out)
    outputs = tf.concat(head_outputs, axis=-1)
    outputs = self.proj(outputs)
    return outputs


In [32]:
# Feed Forward Layer which goes into attn block.
class FFLayer(tf.keras.layers.Layer):
  def __init__(self, n_embed):
    super().__init__()
    self.n_embed = n_embed
    self.net = tf.keras.layers.Dense(4*n_embed,activation="relu") #n_embed
    self.proj = tf.keras.layers.Dense(n_embed)
    self.dropout_layer = tf.keras.layers.Dropout(0.2)


  def call(self, x):
    out = self.net(x)
    out = self.proj(out)
    out = self.dropout_layer(out, training=True)
    return out

In [34]:
class AttnBlock(tf.keras.layers.Layer):
  def __init__(self, n_embed, n_head):
    super().__init__()
    self.head_size = n_embed//n_head
    self.sa = MultiHead(n_head, self.head_size)
    self.ffwd = FFLayer(n_embed)
    self.ln1 = tf.keras.layers.LayerNormalization(axis=-1, input_shape=(n_embed,))
    self.ln2 = tf.keras.layers.LayerNormalization(axis=-1, input_shape=(n_embed,))


  def call(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [40]:
class Transformer :
  def __init__(self):

    self.emb_layer = None
    self.pos_table = None
    self.emb_size = embed_size
    self.pos_size = self.emb_size
    self.num_multi_heads = num_heads

    self.blockone = AttnBlock(self.emb_size, self.num_multi_heads)
    self.blocktwo = AttnBlock(self.emb_size, self.num_multi_heads)
    self.blockthree = AttnBlock(self.emb_size, self.num_multi_heads)
    self.ln = tf.keras.layers.LayerNormalization(axis=-1, input_shape=(self.emb_size,))


  def build_model(self):

        self.emb_layer = tf.keras.layers.Embedding(vocab_size, self.emb_size, mask_zero=True, name="emb_layer")
        self.pos_table = tf.keras.layers.Embedding(block_size, self.pos_size)
        pos_emb = self.pos_table(tf.range(0,8))

        song_input = tf.keras.layers.Input(shape=(8,), dtype=tf.float32, name="Inputblock")
        song_embed = self.emb_layer(song_input)
        cat_input = pos_emb+song_embed

        x = self.blockone(cat_input)
        x = self.blocktwo(x)
        x = self.blockthree(x)
        x = self.ln(x)

        logits = tf.keras.layers.Dense(vocab_size, activation=None, use_bias=False)(x)

        inputs = []
        inputs.append(("Inputblock", song_input))

        inputs = dict(inputs)

        model = tf.keras.Model(inputs=inputs, outputs=logits)

        return model


In [41]:
xb, yb = get_batch('train')

In [42]:
# Lets do 1 forward pass to make sure all is OK.
GPTminiclass = Transformer()
GPTmini = GPTminiclass.build_model()
logits = GPTmini(xb)

In [47]:
# Use Tape Gradient and cat cross entropy for backward pass.

def loss_fn(yb, logits):
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  loss = loss_fn(yb, logits)
  return loss

def trainstep(xb, yb):
  with tf.GradientTape() as tape:
    logits = GPTmini(xb, yb)
    loss = loss_fn(yb, logits)
  grads = tape.gradient(loss, GPTmini.trainable_variables)
  watched_var = [var.name for var in tape.watched_variables()]
  return grads, loss

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)


In [88]:
# Lets train the model
for steps in range(500):
  xb, yb = get_batch('train')
  grads, currLoss = trainstep(xb, yb)
  optimizer.apply_gradients(zip(grads, GPTmini.trainable_variables))

In [89]:
currLoss

<tf.Tensor: shape=(), dtype=float32, numpy=2.073223>

In [57]:
# Inference. Lets test the model.

def generate(idx, max_new_tokens):

      for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]
        logits = GPTmini(idx_cond)

        logits = logits[:,-1,:]
        idx_next = tf.random.categorical(logits, 1)
        idx_next = tf.cast(idx_next, dtype=tf.int32)
        idx = tf.concat([idx, idx_next], axis=1)
      return idx




In [91]:
testStr = "that if " # Seed to generate next sequence
tokenTestStr = encode(testStr)
vectorTestStr = tf.constant(tokenTestStr)
idx = tf.reshape(vectorTestStr, (1,8))

generatedTextTokens = generate(idx, 1000)
numpyText = generatedTextTokens.numpy()
numpyText = numpyText.reshape(-1)

# Convert tokens back to text and print
listRet = decode(list(numpyText))
print(listRet)

that if horent caus beince, har gad, he couinome lmoht mere beistadeptach, wyoht,to the fethel.,,whe how, in scent: lead thouy tak the you sither thestald to foromegoorwithe., whather plive,gode.,,whhe hasrt tich and bara:,hee is dersebay in it broie. ,and thou key med,,a in tope manst thold be
    ical
.lewnfer o mmave hall sisicys:,there, hind twef delt hoveye or ie bince, hear with deent fird my the so f aledbarnd buttany the his. shis,he hinky as meth tome mosse shour monrea,, anevons ket,hen es,for somy ham chiviomath, yould theriphaked giove cell sat bpad,felloss .ditik, kenon likst hand,,offet ath oo, gey,hat his to he sirriifors, woee worth, wearild i habid dep, i and he wisse ghow,i ans, it is then reidsablerle
        i poriie, with me suruesintrus off i n,to thou is nor quages, bus a groarnim an be me friemes dughids wine ird to gawar hank in i the thom me he ose, ond,.,what i:,you, flas baly lonfly tomt tocon moresins:,i what.,,padelabiny, let encerwarl iw ar pind intpeeot: